In [ ]:
import dropbot
import logging; logging.basicConfig(level=logging.DEBUG)
import time

import mr_box_peripheral_board as mrbox
import serial
import numpy as np

#Connect to peripheral
reload(mrbox)

# Try to connect to MR-Box control board.
retry_count = 2
for i in xrange(retry_count):
    try:
        proxy.terminate()
    except NameError:
        pass
    try:
        proxy = mrbox.SerialProxy(baudrate=57600, settling_time_s=2.5)
        break
    except serial.SerialException:
        time.sleep(1)
else:
    raise IOError('Could not connect to MR-Box control board.')
    
proxy._timeout_s = 20

#Connect to dropbot
try:
    db_proxy.terminate()
except:
    pass

db_proxy = dropbot.SerialProxy()
print db_proxy.properties


In [ ]:
# set output voltage to 3.3V
db_proxy.hv_output_selected = False
db_proxy.hv_output_enabled = False

state = np.zeros(db_proxy.number_of_channels)
db_proxy.state_of_channels = state   
state

#Turn of led panel
proxy.led1.on = False
proxy.led2.on = False

In [ ]:
def set_RGB(color, intensity):    
    RGB_Channels = {'Red_Low':24, 'Red_Mid':28, 'Red_High':32, 
                    'Blue_Low':4, 'Blue_Mid':8, 'Blue_High':12, 
                    'Green_Low':44, 'Green_Mid':48, 'Green_High':52}
    
    rgb_c = color + '_'+ intensity
    state[RGB_Channels[rgb_c]] = 1
    db_proxy.state_of_channels = state 
    
def set_LED(led): # 0 to 12
    #LED0 is the top left LED
    LED_Channels = {'LED0':71, 'LED1':119,
               'LED2':79, 'LED3':75, 'LED4':115, 'LED5':111,
               'LED6':83, 'LED7':87, 'LED8':91, 'LED9':103, 'LED10':107,
               'LED11':95, 'LED12':99}
    
    led_c = 'LED'+str(led)
    state[LED_Channels[led_c]] = 1
    db_proxy.state_of_channels = state 

In [ ]:
import time

color = ['Red', 'Blue', 'Green']
intensity = ['Low', 'Mid', 'High']
led = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]



for x in led:
    for y in color:
        for z in intensity:
            #Set the Color, the Intensity and the LED
            set_RGB(y, z)
            set_LED(x)
            time.sleep(1)
            state = np.zeros(db_proxy.number_of_channels) #Turn off all channels
            db_proxy.state_of_channels = state
            time.sleep(0.1)
state = np.zeros(db_proxy.number_of_channels) #Turn off all channels
db_proxy.state_of_channels = state 

In [ ]:
proxy.pin_mode(9, 1)
proxy.pmt_close_shutter()

Vcont = 700
pot_i = int((Vcont/1200.0)*255)
proxy.pmt_set_pot(pot_i)

In [ ]:
from collections import OrderedDict
import pandas as pd

INPUT_RANGE_UNIPOLAR = 1
INPUT_RANGE_BIPOLAR = 2
CLOCK_SOURCE_EXTERNAL = 1
CLOCK_SOURCE_INTERNAL = 2
FORMAT_OFFSET = 1
FORMAT_TWOS_COMPLEMENT = 2
CONVERSION_MODE_SINGLE = 1
CONVERSION_MODE_CONTINUOUS = 2

def MAX11210_begin(proxy):
    proxy.MAX11210_setDefault();
    proxy.MAX11210_setLineFreq(60);  # 60 Hz
    proxy.MAX11210_setInputRange(INPUT_RANGE_UNIPOLAR);
    proxy.MAX11210_setClockSource(CLOCK_SOURCE_INTERNAL);
    proxy.MAX11210_setEnableRefBuf(True);
    proxy.MAX11210_setEnableSigBuf(True);
    proxy.MAX11210_setFormat(FORMAT_OFFSET);
    proxy.MAX11210_setConvMode(CONVERSION_MODE_SINGLE);
    proxy.MAX11210_selfCal();
#     proxy.MAX11210_sysOffsetCal();
    proxy.MAX11210_sysGainCal();

MAX11210_begin(proxy)
calibration_settings = \
    pd.Series(OrderedDict([('SelfCalGain', proxy.MAX11210_getSelfCalGain()),
                           ('SelfCalOffset', proxy.MAX11210_getSelfCalOffset()),
                           ('SysGainCal', proxy.MAX11210_getSysGainCal()),
                           ('SysOffsetCal', proxy.MAX11210_getSysOffsetCal())]))
print '# Calibration settings #\n'
print calibration_settings

In [ ]:
import time

color = ['Red', 'Blue', 'Green']
intensity = ['Low', 'Mid', 'High']
led = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

rate = 120
max_range = 20

proxy.MAX11210_setConvMode(1)
proxy.pmt_open_shutter()

bkg = []

#Measure Background
for i in range (0,max_range):
    proxy.MAX11210_setRate(rate)
    bkg_i = proxy.MAX11210_getData()
    bkg.append(bkg_i)
background = sum(bkg)/len(bkg)
background /= (((2 ** 24 - 1)/(3.0))*30e3)
print 'Background: %s' %background

dtype = [('Color','str'), ('Intensity','str'), ('Electrode','int32'), ('Signal','float32'),('X','float32'),('Y','float32')]
values = np.zeros(13*3*3, dtype=dtype)
df = pd.DataFrame(values)
enumerator = 0

x_pos=[3,4,2,3,4,5,1,2,3,4,5,3,4]
y_pos=[4,4,3,3,3,3,2.5,2,2,2,2,1,1]

R_x_pos = [2.75, 4.25, 1.75, 2.75, 4.25, 5.25, 0.75, 1.75, 2.75, 4.25, 5.25, 2.75, 4.25]
R_y_pos = [3.75, 3.75, 3.25, 3.25, 3.25, 3.25, 2.75, 1.75, 1.75, 1.75, 1.75, 1.25, 1.25]

B_x_pos = [2.75, 3.75, 2.25, 3.25, 4.25, 5.25, 1.25, 1.75, 2.75, 3.75, 4.75, 3.25, 4.25]
B_y_pos = [4.25, 3.75, 3.25, 3.25, 2.75, 2.75, 2.75, 2.25, 2.25, 1.75, 1.75, 1.25, 0.75]

G_x_pos = [3.25, 3.75, 2.25, 3.25, 3.75, 4.75, 1.25, 2.25, 3.25, 3.75, 4.75, 3.25, 3.75]
G_y_pos = [4.25, 4.25, 2.75, 2.75, 2.75, 2.75, 2.25, 2.25, 2.25, 2.25, 2.25, 0.75, 0.75]



for x in led:
    for y in color:
        for z in intensity:
            state = np.zeros(db_proxy.number_of_channels) #Turn off all channels
            db_proxy.state_of_channels = state
            time.sleep(0.1)
            #Read once to clear the ADC
            proxy.MAX11210_setRate(rate)
            proxy.MAX11210_getData()
            #Set the Color, the Intensity and the LED
            set_RGB(y, z)
            set_LED(x)
            readings = []
            for i in range (0,max_range):
                proxy.MAX11210_setRate(rate)
                reading_i = proxy.MAX11210_getData()
                readings.append(reading_i)
            signal = sum(readings)/len(readings)
            signal /= (((2 ** 24 - 1)/(3.0))*30e3)
            df.loc[enumerator, 'Color'] = y
            df.loc[enumerator, 'Intensity'] = z
            df.loc[enumerator, 'Electrode'] = x
            df.loc[enumerator, 'Signal'] = signal
            if (y == 'Red'):
                df.loc[enumerator, 'X'] = R_x_pos[x]
                df.loc[enumerator, 'Y'] = R_y_pos[x]
            elif (y == 'Blue'):
                df.loc[enumerator, 'X'] = B_x_pos[x]
                df.loc[enumerator, 'Y'] = B_y_pos[x]
            elif (y == 'Green'):
                df.loc[enumerator, 'X'] = G_x_pos[x]
                df.loc[enumerator, 'Y'] = G_y_pos[x]
            enumerator += 1
            #print 'Color: %s , Intensity: %s, Electrode: %s, Signal: %s' %(y , z, x, signal)

            
proxy.pmt_close_shutter()
state = np.zeros(db_proxy.number_of_channels) #Turn off all channels
db_proxy.state_of_channels = state 


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D

df['Signal_N'] = 0

X_axis = df.loc[:,'X']  
Y_axis = df.loc[:,'Y']
df.loc[df['Color'] == 'Red', 'Signal_N'] = (df.loc[df['Color'] == 'Red', 'Signal']/ 
                                            df.loc[df['Color'] == 'Red', 'Signal'].max())
df.loc[df['Color'] == 'Blue', 'Signal_N'] = (df.loc[df['Color'] == 'Blue', 'Signal']/ 
                                            df.loc[df['Color'] == 'Blue', 'Signal'].max())
df.loc[df['Color'] == 'Green', 'Signal_N'] = (df.loc[df['Color'] == 'Green', 'Signal']/ 
                                            df.loc[df['Color'] == 'Green', 'Signal'].max())
Z_axis = df.loc[:,'Signal_N'] # Normalize Signal

dz = np.zeros_like(Z_axis)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
col= cm.rainbow(Z_axis)
ax.bar3d(X_axis, Y_axis, dz, 0.25, 0.25,Z_axis, color=col)
ax.set_xlim([0,6])
ax.set_xlim([0,6])
ax.view_init(90, -90)

sm = plt.cm.ScalarMappable(cmap=plt.get_cmap("rainbow"), norm=plt.Normalize(vmin=0, vmax=1))
# fake up the array of the scalar mappable. Urgh...
sm._A = []
plt.colorbar(sm)

ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Average Signal, A')

Xx=[]
Yy=[]
w = 0.45

for i in range(0,len(x_pos)):          
    Xx =[(x_pos[i]-w),(x_pos[i]-w),(x_pos[i]+w),(x_pos[i]+w),(x_pos[i]-w)]
    Yy =[(y_pos[i]-w),(y_pos[i]+w),(y_pos[i]+w),(y_pos[i]-w),(y_pos[i]-w)]
    plt.plot(Xx, Yy, color='0')
    Xx=[]
    Yy=[]

In [ ]:
df.to_csv('PMT_Analysis_DRC_1.csv')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax2 = fig.add_subplot(111)
ax3 = fig.add_subplot(111)
ax4 = fig.add_subplot(111)

Xx=[]
Yy=[]
w = 0.45

for i in range(0,len(x_pos)):          
    Xx =[(x_pos[i]-w),(x_pos[i]-w),(x_pos[i]+w),(x_pos[i]+w),(x_pos[i]-w)]
    Yy =[(y_pos[i]-w),(y_pos[i]+w),(y_pos[i]+w),(y_pos[i]-w),(y_pos[i]-w)]
    ax1.plot(Xx, Yy, color='0')
    Xx=[]
    Yy=[]

for i in range(1,len(x_pos)+1):
    x1= R_x_pos[:i]
    y1= R_y_pos[:i]
    ax2.scatter(x1, y1)
    
    x2= B_x_pos[:i]
    y2= B_y_pos[:i]
    ax3.scatter(x2, y2)
    
    x3= G_x_pos[:i]
    y3= G_y_pos[:i]
    ax4.scatter(x3, y3)